# Referência - Curso de Aprendizado de Máquinas que fiz na Escola de Férias.




# Introdução


Neste pequeno projeto, é ilustrado a aplicação de alguns modelos de Machine Learning, tais como  Regressão Logística, kNN, Decision Tree, Random Forest e Support Vector Machine, em um problema envolvendo pacientes com problemas cardíacos. O objetivo é criar um modelo capaz de "prever" se um determinado indivíduo virá a óbito.
Os dados são do artigo: *Davide Chicco, Giuseppe Jurman: Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making 20, 16 (2020).* Você encontra  [aqui.](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5)







Começamos importando o pandas e fixando a semente aleatória

In [1]:
import pandas as pd
seed = 0 # Fixar a semente aleatória dos algoritmos

Vamos carregar o dataset e visualizar o conjunto de dados. 


*   É legal visualizar o dataset para verificar um pouco da sua estrutura.
*   Existem outras maneiras mais espertas de carregar um dataset. Poderia ser por exemplo à partir da própria internet, através de um link http.





In [2]:
data = pd.read_csv('heart_failure.csv')
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


Você pode verificar com mais detalhes o significado de cada um dos atributos aqui nesse [link](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5/tables/1).

Vamos escrever na matriz X as features e no vetor Y os rótulos associados. 




In [3]:
X = data.iloc[:, 0:-1].values #features: da primeira até a penúltima coluna
y = data.iloc[:, -1].values #rótulo: última coluna

Começamos separando o conjunto de treino e teste O pré-processamento, se feito no conjunto completo, não pode introduzir  informações futura no conjunto de treino. É importante salientar que se você deseja uma estimativa do erro de generalização, que é dado em termos de uma probabilidade independente do conjunto de treino, você deve usar o conjunto de teste uma única vez!

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 1/3, 
                                                    random_state = seed)

Separar o conjunto de treino em duas partes, uma será usada para validação. Faremos isso pois queremos treinar alguns modelos e verificar qual deles é o melhor candidato para generalização. Dito isto, dado que só devemos usar o 
conjunto de teste uma única vez, devemos fazer a decisão da escolha do 
melhor modelo "no próprio conjunto de treino". 

In [5]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,
                                                      y_train, 
                                                      test_size = 1/3, 
                                                      random_state = seed)

Tipicamente os algoritmos necessitam que todas as características estejam em faixas similares de intervalos, por isso a importância de colocar todos os atributos na mesma escala.

In [6]:
# Feature Scaling: alguns algoritmos são sensíveis à escala
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train_2)
X_val = scaler.transform(X_val)

Agora vamos avaliar diversos modelos para encontrar o melhor candidato. 
Faremos isso da maneira "feia" apenas por uma questão de simplicidade.

In [7]:
#Dicionário de resultados
dic = {}

Para comparar os resultados vamos usar a acurácia


In [8]:
from sklearn.metrics import accuracy_score

# Regressão Logística


In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = seed)
lr.fit(X_train_2, y_train_2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# Avaliando a regressão logística e anotando no dicionário
dic["Logistic_regression"] = accuracy_score(y_val, lr.predict(X_val))

# KNN


In [11]:
# Treinando o k-NN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train_2, y_train_2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [12]:
# Avaliando o k-NN e anotando no dicionário
dic["kNN_7"] =  accuracy_score(y_val, knn.predict(X_val))

# Decision Tree


In [13]:
#Treinando uma árvore de decisão
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 4, random_state = seed)
tree.fit(X_train_2, y_train_2)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [14]:
# Avaliando a árvore e anotando no dicionário
dic["Decision_tree_4"] = accuracy_score(y_val, tree.predict(X_val))

# Random Forest


In [15]:
#Treinando uma floresta aleatória
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 100, 
                            random_state = seed)

In [16]:
# Avaliando a florest aleatória e anotando no dicionário
rf.fit(X_train_2, y_train_2)
dic["Random_forest_100"] = accuracy_score(y_val, rf.predict(X_val))

# Linear Support Vector Machine

In [17]:
from sklearn.svm import LinearSVC
svc = LinearSVC()

In [18]:
#Avaliando o support vector machine e anotando no dicionário
svc.fit(X_train_2, y_train_2)
dic["SVC"] = accuracy_score(y_val, svc.predict(X_val))

# Avaliando modelos


In [19]:
validation = pd.Series(dic, name="Acurácia")

In [20]:
validation

Logistic_regression    0.791045
kNN_7                  0.671642
Decision_tree_4        0.701493
Random_forest_100      0.776119
SVC                    0.761194
Name: Acurácia, dtype: float64

# Treinando o modelo final


Uma vez definido o melhor modelo, que deverá ser decidido usando **somente o conjunto de treinamento**, podemos utilizar o conjunto de teste para estimar o erro de generalização.

In [21]:
#Treinando o melhor modelo no conjunto inteiro:
scaler = MinMaxScaler()    
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  

Aqui você deve usar o algoritmo que demonstrou possuir melhor performance

In [22]:
lr = LogisticRegression(random_state = seed)     
lr.fit(X_train, y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
print("Acurácia final: ", str(accuracy_score(y_test, lr.predict(X_test))))

Acurácia final:  0.79
